<a href="https://colab.research.google.com/github/7benson/SMS-Spam-Classifier/blob/main/SMS_spam_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install --upgrade --no-deps --force-reinstall kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d uciml/sms-spam-collection-dataset

Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
sms-spam-collection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [26]:
file_name='sms-spam-collection-dataset.zip'
from zipfile import ZipFile
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Extraction Done')

Extraction Done


In [27]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [28]:
import pandas as pd
df=pd.read_csv('./spam.csv',usecols=['v1','v2'], encoding='latin-1')
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [40]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
corpus=[]
sentences=df.v2
stemmer=PorterStemmer()
for i in range(len(sentences)):
  sent=re.sub('[^A-Za-z]',' ',sentences[i])
  sent=sent.lower()
  words=nltk.word_tokenize(sent)
  words=[stemmer.stem(word) for word in words if word not in stopwords.words('english')]
  sent=' '.join(words)
  corpus.append(sent)
y=pd.get_dummies(df.v1,drop_first=True)

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
corpus=cv.fit_transform(corpus).toarray()

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.2,random_state=42)

from sklearn.linear_model import LogisticRegression
logistic_regression=LogisticRegression()
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9775784753363229
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.83      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

0.9090909090909091
[[965   0]
 [ 25 125]]


In [66]:
y

,spam
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [30]:
from sklearn.naive_bayes import GaussianNB
logistic_regression=GaussianNB()
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8618834080717489
              precision    recall  f1-score   support

           0       0.98      0.86      0.91       965
           1       0.49      0.89      0.64       150

    accuracy                           0.86      1115
   macro avg       0.74      0.88      0.77      1115
weighted avg       0.92      0.86      0.88      1115

0.6350710900473934
[[827 138]
 [ 16 134]]


In [31]:
import tensorflow as tf

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=70, activation='relu'))
ann.add(tf.keras.layers.Dense(units=80, activation='relu'))
ann.add(tf.keras.layers.Dense(units=40, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# ann.fit(X_train, y_train, batch_size = 32, epochs = 100,validation_data=(X_test, y_test))
ann.fit(corpus, y, batch_size = 32, epochs = 5,validation_split=0.2)
# corpus,y

Epoch 1/5
140/140 [==============================] - 1s 5ms/step - loss: 0.2427 - accuracy: 0.9197 - val_loss: 0.0548 - val_accuracy: 0.9794
Epoch 2/5
140/140 [==============================] - 0s 3ms/step - loss: 0.0272 - accuracy: 0.9933 - val_loss: 0.0550 - val_accuracy: 0.9865
Epoch 3/5
140/140 [==============================] - 0s 3ms/step - loss: 0.0069 - accuracy: 0.9980 - val_loss: 0.0682 - val_accuracy: 0.9830
Epoch 4/5
140/140 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.0880 - val_accuracy: 0.9830
Epoch 5/5
140/140 [==============================] - 0s 3ms/step - loss: 9.5643e-04 - accuracy: 1.0000 - val_loss: 0.1009 - val_accuracy: 0.9821


In [32]:
ann.save('sms-spam-classifier.h5')

In [33]:
model = tf.keras.models.load_model('sms-spam-classifier.h5')


In [76]:
import pandas as pd
df=pd.read_csv('./spam.csv',usecols=['v1','v2'], encoding='latin-1')

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
corpus=[]
sentences=list(df.v2)
print(type(sentences))
sentences.append("Congratulations you have won a prize")
print(sentences[-1])
stemmer=PorterStemmer()
for i in range(len(sentences)):
  sent=re.sub('[^A-Za-z]',' ',sentences[i])
  sent=sent.lower()
  words=nltk.word_tokenize(sent)
  words=[stemmer.stem(word) for word in words if word not in stopwords.words('english')]
  sent=' '.join(words)
  corpus.append(sent)
y=pd.get_dummies(df.v1,drop_first=True)

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(ngram_range=(1,2),max_features=5000)
corpus=cv.fit_transform(corpus).toarray()


<class 'list'>
Congratulations you have won a prize


In [79]:
from sklearn.naive_bayes import MultinomialNB

# 2. instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()

nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.9775784753363229
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       965
           1       0.90      0.94      0.92       150

    accuracy                           0.98      1115
   macro avg       0.94      0.96      0.95      1115
weighted avg       0.98      0.98      0.98      1115

0.9185667752442996
[[949  16]
 [  9 141]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [81]:
from sklearn.externals import joblib
joblib.dump(nb,'multinomial-db.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['multinomial-db.pkl']

In [80]:
import numpy as np
y_temp=corpus[-1]
temp=np.array(y_temp).reshape((1,5000))
print(model.predict(temp))
print(nb.predict(temp))
logistic_regression.predict(temp)
# temp.shape

[[0.02430574]]
[1]


array([0], dtype=uint8)

In [61]:
model.input_shape

(None, 5000)